In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # Input: 28x28 pixels, Output: 128 neurons
        self.fc2 = nn.Linear(128, 10)  # Output: 10 classes (digits 0-9)

    def forward(self, x):
        x = x.view(-1, 28*28)  # Flatten the input
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop with benchmarking
epochs = 5
for epoch in range(epochs):
    start_time = time.time()  # Start time for training
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    end_time = time.time()  # End time for training
    training_time = end_time - start_time
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}, Time: {training_time:.4f}s")

# Evaluate the model on the test set and benchmark the accuracy
correct = 0
total = 0
start_time = time.time()  # Start time for testing
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

end_time = time.time()  # End time for testing
testing_time = end_time - start_time
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%, Testing Time: {testing_time:.4f}s")


100%|██████████| 9.91M/9.91M [00:01<00:00, 6.05MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 160kB/s]
100%|██████████| 1.65M/1.65M [00:01<00:00, 1.51MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.15MB/s]


Epoch [1/5], Loss: 0.3277, Time: 13.8831s
Epoch [2/5], Loss: 0.1509, Time: 13.8539s
Epoch [3/5], Loss: 0.3818, Time: 13.7656s
Epoch [4/5], Loss: 0.4637, Time: 14.3902s
Epoch [5/5], Loss: 0.1891, Time: 18.0007s
Test Accuracy: 92.53%, Testing Time: 2.0102s


In [3]:
# Strong LLM
import time
import jax
import jax.numpy as jnp
import numpy as np
import tensorflow_datasets as tfds
import optax
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense, Relu, Flatten

# ---------------------------
# Data Loading and Preprocessing
# ---------------------------
def preprocess(example):
    # Convert image to float32, scale to [0,1] then normalize to [-1,1] (like (x-0.5)/0.5)
    image = np.array(example['image'], dtype=np.float32) / 255.0
    image = (image - 0.5) / 0.5
    label = example['label']
    return image, label

def dataset_to_batches(ds, batch_size):
    # Convert the TFDS dataset to numpy arrays and create batches.
    ds = tfds.as_numpy(ds)
    images, labels = [], []
    for example in ds:
        img, lab = preprocess(example)
        images.append(img)
        labels.append(lab)
    images = np.stack(images)
    labels = np.array(labels)
    num_batches = images.shape[0] // batch_size
    batches = []
    for i in range(num_batches):
        batch_images = images[i*batch_size:(i+1)*batch_size]
        batch_labels = labels[i*batch_size:(i+1)*batch_size]
        batches.append((batch_images, batch_labels))
    return batches

batch_size = 64
train_ds = tfds.load('mnist', split='train', shuffle_files=True)
test_ds  = tfds.load('mnist', split='test',  shuffle_files=False)

train_batches = dataset_to_batches(train_ds, batch_size)
test_batches  = dataset_to_batches(test_ds, batch_size)

# ---------------------------
# Model Definition using stax
# ---------------------------
# Define a simple neural network that mirrors the PyTorch model:
# - Flatten the input (28x28 or 28x28x1)
# - Dense layer with 128 units and ReLU activation
# - Dense layer with 10 outputs (for the 10 classes)
init_random_params, predict = stax.serial(
    Flatten,
    Dense(128),
    Relu,
    Dense(10)
)

# Initialize model parameters. The expected input shape is (batch, 28, 28, 1).
rng = jax.random.PRNGKey(0)
_, params = init_random_params(rng, (-1, 28, 28, 1))

# ---------------------------
# Loss Function and Optimizer
# ---------------------------
# Define the cross-entropy loss function. Note that optax.softmax_cross_entropy
# expects logits and one-hot encoded labels.
def loss_fn(params, batch):
    images, labels = batch
    logits = predict(params, images)
    one_hot = jax.nn.one_hot(labels, num_classes=10)
    loss = optax.softmax_cross_entropy(logits, one_hot).mean()
    return loss

# Use SGD optimizer with learning rate 0.01
optimizer = optax.sgd(learning_rate=0.01)
opt_state = optimizer.init(params)

# Define a single training step with JIT compilation.
@jax.jit
def train_step(params, opt_state, batch):
    loss, grads = jax.value_and_grad(loss_fn)(params, batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss

# ---------------------------
# Training Loop with Benchmarking
# ---------------------------
epochs = 5
for epoch in range(epochs):
    start_time = time.time()
    for batch in train_batches:
        params, opt_state, loss = train_step(params, opt_state, batch)
    end_time = time.time()
    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss:.4f}, Time: {end_time - start_time:.4f}s")

# ---------------------------
# Evaluation on Test Set with Benchmarking
# ---------------------------
correct = 0
total = 0
start_time = time.time()
for batch in test_batches:
    images, labels = batch
    logits = predict(params, images)
    predictions = jnp.argmax(logits, axis=1)
    correct += int(jnp.sum(predictions == labels))
    total += images.shape[0]
end_time = time.time()
accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%, Testing Time: {end_time - start_time:.4f}s")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.FLLWQB_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.FLLWQB_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
Epoch [1/5], Loss: 0.2275, Time: 1.3489s
Epoch [2/5], Loss: 0.1820, Time: 0.7713s
Epoch [3/5], Loss: 0.1566, Time: 0.7866s
Epoch [4/5], Loss: 0.1349, Time: 0.7789s
Epoch [5/5], Loss: 0.1185, Time: 0.9486s
Test Accuracy: 93.76%, Testing Time: 0.6540s


In [4]:
## Weak LLM
import jax.numpy as jnp  # MODIFIED
import jax.random as random  # MODIFIED
import optax  # MODIFIED
import time  # MODIFIED
from flax import linen as nn  # MODIFIED

def generate_random_numbers(key, shape):
    """Generates random numbers using a JAX random key.

    Args:
        key: A JAX random key.
        shape: The shape of the output random array.

    Returns:
        A JAX array of random numbers.
    """
    return random.normal(key, shape)  # MODIFIED

def main():
    """Main function to test the accuracy of a model."""
    # Assuming test_labels and some model output predictions exist
    test_labels = jnp.array([1, 0, 1, 1, 0])  # Example test labels
    predicted_classes = jnp.array([1, 0, 1, 0, 0])  # Example predictions

    start_time = time.time()  # Start time for testing

    # Calculate accuracy
    total = len(test_labels)  # MODIFIED
    correct = jnp.sum(predicted_classes == test_labels)

    end_time = time.time()  # End time for testing
    testing_time = end_time - start_time
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%, Testing Time: {testing_time:.4f}s")  # MODIFIED

if __name__ == "__main__":  # MODIFIED
    main()  # MODIFIED

Test Accuracy: 80.00%, Testing Time: 0.0515s


In [ ]:
"""Error Code
from jax.experimental import stax
from jax.experimental.stax import Dense, Relu, Flatten
Error
Can't import stax from jax

Fix guide
jax.experimental.stax was moved to jax.example_libraries.stax in in JAX v0.2.25
and the deprecated alias was removed in JAX v0.3.16.

Correct code
from jax.example_libraries import stax
from jax.example_libraries.stax import Dense, Relu, Flatten

"""

"""
Error Code:
for batch in train_batches:
  params, opt_state, loss = train_step(params, opt_state, batch, model, optimizer)

Error:
TypeError: Error interpreting argument to <function train_step at 0x7b90a9e7a660>
as an abstract array. The problematic value is of type <class '__main__.SimpleNN'>
and was passed to the function at path model.
This typically means that a jit-wrapped function was called with a
non-array argument, and this argument was not marked as static using the
static_argnums or static_argnames parameters of jax.jit.

Fix guide
modify the @jax.jit decorator for the train_step function to include model and
optimizer as static arguments.
"""

"""Error Code
@jax.jit(static_argnames=('model', 'optimizer'))
def train_step(params, opt_state, batch, model, optimizer):
  loss, grads = jax.value_and_grad(loss_fn)(params, model, batch)

Error:
TypeError: jit() missing 1 required positional argument: 'fun'

Fix guide:
fix this by using functools.partial to supply the static arguments.

Correct Code
from functools import partial
@partial(jax.jit, static_argnames=('model', 'optimizer'))
def train_step(params, opt_state, batch, model, optimizer):
"""

In [12]:
"""This is a version of JAX that provides more comprehensive output and
implement more utilities and complete translation that recreates the full
pipeline of the original PyTorch code (data loading, model definition,
training loop, and evaluation), but it does so using JAX’s functional
style with Flax for the model, optax for optimization, and TFDS for data.

Error code will be recorded from this code version (if applicable)"""

import time
import jax
import jax.numpy as jnp
import numpy as np
import tensorflow_datasets as tfds
import optax
from flax import linen as nn
from functools import partial

# ---------------------------
# Data Loading and Preprocessing
# ---------------------------
def preprocess(example):
    # Convert image to float32, scale to [0,1] then normalize to [-1,1]
    image = np.array(example['image'], dtype=np.float32) / 255.0
    image = (image - 0.5) / 0.5
    # Ensure image shape is (28, 28, 1)
    if image.ndim == 2:
        image = np.expand_dims(image, -1)
    label = example['label']
    return image, label

def get_datasets(batch_size=64):
    # Load MNIST using TensorFlow Datasets
    train_ds = tfds.load('mnist', split='train', shuffle_files=True)
    test_ds  = tfds.load('mnist', split='test',  shuffle_files=False)

    # Convert training dataset to numpy arrays and create batches
    train_images, train_labels = [], []
    for example in tfds.as_numpy(train_ds):
        img, lab = preprocess(example)
        train_images.append(img)
        train_labels.append(lab)
    train_images = np.stack(train_images)
    train_labels = np.array(train_labels)

    # Convert test dataset to numpy arrays and create batches
    test_images, test_labels = [], []
    for example in tfds.as_numpy(test_ds):
        img, lab = preprocess(example)
        test_images.append(img)
        test_labels.append(lab)
    test_images = np.stack(test_images)
    test_labels = np.array(test_labels)

    # Create batches
    train_batches = [(train_images[i:i+batch_size], train_labels[i:i+batch_size])
                     for i in range(0, len(train_labels), batch_size)]
    test_batches = [(test_images[i:i+batch_size], test_labels[i:i+batch_size])
                    for i in range(0, len(test_labels), batch_size)]

    return train_batches, test_batches

# ---------------------------
# Model Definition using Flax Linen
# ---------------------------
class SimpleNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        # Flatten the input (28x28 pixels becomes 784)
        x = x.reshape((x.shape[0], -1))
        x = nn.Dense(features=128)(x)
        x = nn.relu(x)
        x = nn.Dense(features=10)(x)
        return x

def create_train_state(rng, learning_rate):
    model = SimpleNN()
    # Initialize parameters with dummy input: shape (1, 28, 28, 1)
    dummy_input = jnp.ones((1, 28, 28, 1))
    params = model.init(rng, dummy_input)
    # Use SGD optimizer with learning rate 0.01
    optimizer = optax.sgd(learning_rate)
    opt_state = optimizer.init(params)
    return model, params, optimizer, opt_state

# ---------------------------
# Loss Function and Training Step
# ---------------------------
def loss_fn(params, model, batch):
    images, labels = batch
    logits = model.apply(params, images)
    one_hot = jax.nn.one_hot(labels, num_classes=10)
    loss = optax.softmax_cross_entropy(logits, one_hot).mean()
    return loss

# @jax.jit
@partial(jax.jit, static_argnames=('model', 'optimizer'))
def train_step(params, opt_state, batch, model, optimizer):
    loss, grads = jax.value_and_grad(loss_fn)(params, model, batch)
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    return params, opt_state, loss

# ---------------------------
# Main Training and Evaluation Loop
# ---------------------------
def main():
    batch_size = 64
    epochs = 5
    # Load datasets
    train_batches, test_batches = get_datasets(batch_size)

    # Initialize model and optimizer
    rng = jax.random.PRNGKey(0)
    model, params, optimizer, opt_state = create_train_state(rng, learning_rate=0.01)

    # Training Loop with Benchmarking
    for epoch in range(epochs):
        start_time = time.time()
        for batch in train_batches:
            params, opt_state, loss = train_step(params, opt_state, batch, model, optimizer)
        end_time = time.time()
        training_time = end_time - start_time
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss:.4f}, Time: {training_time:.4f}s")

    # Evaluation on Test Set with Benchmarking
    correct = 0
    total = 0
    start_time = time.time()
    for batch in test_batches:
        images, labels = batch
        logits = model.apply(params, images)
        predictions = jnp.argmax(logits, axis=1)
        correct += int(jnp.sum(predictions == labels))
        total += images.shape[0]
    end_time = time.time()
    testing_time = end_time - start_time
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%, Testing Time: {testing_time:.4f}s")

if __name__ == "__main__":
    main()


Epoch [1/5], Loss: 0.3185, Time: 1.7338s
Epoch [2/5], Loss: 0.2485, Time: 0.7743s
Epoch [3/5], Loss: 0.2204, Time: 0.8832s
Epoch [4/5], Loss: 0.2031, Time: 1.3212s
Epoch [5/5], Loss: 0.1907, Time: 1.1907s
Test Accuracy: 93.49%, Testing Time: 1.8806s
